Ноутбук для обучения моделей на текстах, которые могут иметь разное количесто содержащейся пунктуации.

Модели и датасеты которые есть на данный момент:
Новые модели - это модели 1, доученные на второй трети датасетов

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dataset_names = ['only_words', 'main_punct', 'main_punct_plus', 'initial']
dataset_paths = ['/content/drive/MyDrive/QA_training/datasets_csv/ms_only_words_train.csv',
                 '/content/drive/MyDrive/QA_training/datasets_csv/ms_main_punct_train.csv',
                 '/content/drive/MyDrive/QA_training/datasets_csv/ms_main_punct_plus_train.csv',
                 '/content/drive/MyDrive/QA_training/datasets_csv/ms_initial_train.csv']

In [3]:
model_paths = ['/content/drive/MyDrive/QA_training/models/only_words_bert_model_1',
               '/content/drive/MyDrive/QA_training/models/main_punct_bert_model_1',
               '/content/drive/MyDrive/QA_training/models/main_punct_plus_bert_model_1',
               '/content/drive/MyDrive/QA_training/models/initial_bert_model_1']
new_model_names = ['only_words_bert_model_2',
                   'main_punct_bert_model_2',
                   'main_punct_plus_bert_model_2',
                   'initial_bert_model_2']

# Тренировка

In [4]:
!pip install import-ipynb
%cd '/content/drive/My Drive/QA_training/notebooks'
import import_ipynb
from my_functions import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=cf25c64fe48e274c93edc7ce38ad24c2028dcbb32f483cc082c673ca29dd2959
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/My Drive/QA_training/notebooks
importing Jupyter notebook from my_functions.ipynb
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wra9260z
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-wra9260z
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 12.0MB/s 
     |████████████████████████████████| 890kB 59.7MB/s 
  Created wheel for transformers: filename=transformers-4.2.0.dev0-cp36-none-any.whl size=1527266 sha256=7cfb9e3b47fd7fe8789eb8dbeade4e63edb1d3f39dd59474db4eaaafaf12ec94
  Stor

In [5]:
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [6]:
number = 3

In [7]:
train_csv_path = dataset_paths[number]
print("Dataset directory:", train_csv_path)

train_dataset = prepare_and_merge_data(train_csv_path, tokenizer, 'msmarco', text_num=1, parts_num = 3, part = 2,  text_ids = None, return_answers = False)

Dataset directory: /content/drive/MyDrive/QA_training/datasets_csv/ms_initial_train.csv


In [9]:
dir = "/content/drive/My Drive/QA_training/" + new_model_names[number]
print('Model directory:', dir)

#model = BertForQuestionAnswering.from_pretrained("bert-base-uncased");
#model = BertForQuestionAnswering.from_pretrained(model_paths[number]);
model =  BertForQuestionAnswering.from_pretrained(dir+'/checkpoint-24000');

Model directory: /content/drive/My Drive/QA_training/initial_bert_model_2


In [10]:
training_args = TrainingArguments(
    output_dir=dir,                  # output directory
    learning_rate=2e-5,          
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir=dir+'/logs',            # directory for storing logs
    logging_steps=250,
    prediction_loss_only=False,
    label_names = ["start_positions", "end_positions"],
    overwrite_output_dir = True
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    #eval_dataset=val_dataset,            # evaluation dataset

)

In [12]:
#trainer.train()
trainer.train(dir+'/checkpoint-24000')
trainer.save_model(dir)

Step,Training Loss
24250,0.012722
24500,1.222908
24750,1.198373
25000,1.216693
25250,1.223840
25500,1.244268
25750,1.209413
26000,1.193915
26250,1.180130
26500,1.227783
